In [4]:
!pip install PyPDF2
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install higgingface-hub
!pip install pypdf
!pip install accelerate
!pip install llama-cpp-python
!pip install git+https://github.com/huggingface/transformers

ERROR: Could not find a version that satisfies the requirement higgingface-hub (from versions: none)
ERROR: No matching distribution found for higgingface-hub
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jhoo8pek
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jhoo8pek
  Resolved https://github.com/huggingface/transformers to commit 6cbc1369a330860c128a1ba365f246751382c9e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from PyPDF2 import PdfReader


In [6]:
pdf_reader = PdfReader('/content/drive/MyDrive/Atomic Habits James Clear.pdf')
text = ''
for page in pdf_reader.pages:
  text += page.extract_text()
print(text)

AN IMPRINT OF PENGUIN RAND OM HOUSE LLC
375 Hudson Street
New Y ork, New Y ork 10014
Copyright © 2018 by James Clear
Penguin supports copyright. Copyright fuels creativity , encourages diverse voices, promotes free speech, and
creates a vibrant culture. Thank you for buying an authorized edition of this book and for complying with
copyright laws by not reproducing, scanning, or distributing any part of it in any form without permission. Y ou
are supporting writers and allowing Penguin to continue to publish books for every reader .
Ebook ISBN 978073521 1308
While the author has made every ef fort to provide accurate Internet addresses at the time of publication,
neither the publisher nor the author assumes any responsibility for errors, or for changes that occur after
publication. Further , the publisher does not have any control over and does not assume any responsibility for
author or third-party websites or their content.
Version_1a · t o m · i c
əˈtämik
1. an extremely small amount

In [7]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200,length_function=len)
chunks = text_splitter.split_text(text)

In [8]:
len(chunks)

572

In [9]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [11]:
vector_store=FAISS.from_texts(chunks,embedding=embeddings)

In [10]:
llm=LlamaCpp(streaming=True,
             model_path='/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
             temperature=0.75,
             top_p=1,
             verbose=True,
             n_ctx=4096)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [12]:
from langchain.schema import retriever
qa=RetrievalQA.from_chain_type(llm=llm,
                               chain_type='stuff',
                               retriever=vector_store.as_retriever(search_kwargs={'k':2}))

In [13]:
query='what is breakthrough moments'

In [14]:
qa.run(query)

' Breakthrough moments are those moments that result from many previous actions and build up the potential required to unleash a major change.'

In [19]:
import sys
while True:
  user_input=input(f'Input Prompt')
  if user_input=='exit':
    print('Exiting')
    sys.exit()
  if user_input=='':
    continue
  result=qa({'query':user_input})
  print(f'Answer: {result}')

Llama.generate: prefix-match hit


Answer: {'query': 'what is breakthrough moments', 'result': ' Breakthrough moments are significant moments of change that result from many previous actions and build up potential for a major change. They occur when a small change in circumstances triggers a large and sudden shift in behavior or outcomes. Breakthrough moments can be found in various fields, including cancer research, bamboo growth, and personal fitness.'}
Input Promptexit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
